In [ ]:
import pandas as pd
!gdown 1wvqGslWLauKyZxLY-d3RQCTyHCb5wJCO
df = pd.read_csv('train.csv')
df.head()
# 2. Разделение признаков и целевой переменной
X = df.drop('Exited', axis=1)
y = df['Exited']
y.head()
import os

file_size_bytes = os.path.getsize('train.csv')
file_size_gb = file_size_bytes / (1024**3)
print(f"Размер файла train.csv: {file_size_gb:.4f} ГБ")
# Сколько строк и столбцов
# ваш код

print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")
# ваш код

# выбираем только числовые столбцы
num_cols = df.select_dtypes(include='number').columns

# мин, макс, среднее и квартели по числовым
summary = df[num_cols].describe().T
summary['median'] = df[num_cols].median()

# отберем только нужные колонки
summary = summary[['min', 'max', 'mean', 'median']]

print(summary)
# ваш код

results = []

for col in df.columns:
    mode_val = df[col].mode()[0]
    unique_count = df[col].nunique()
    results.append({'Признак': col, 'Мода': mode_val,
                   'Уникальных значений': unique_count})

summary_df = pd.DataFrame(results)
print(summary_df)
# доля 1
y.value_counts(normalize=True)
# ваш код

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
numeric_cols = df.select_dtypes(include='number').columns
df_numeric = df[numeric_cols]

outliers = {}

for col in df_numeric.columns:
    Q1 = df_numeric[col].quantile(0.25)
    Q3 = df_numeric[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_mask = (df_numeric[col] < lower_bound) | (
        df_numeric[col] > upper_bound)
    outliers[col] = outlier_mask.sum()

outlier_df = pd.DataFrame.from_dict(
    outliers, orient='index', columns=['Количество выбросов'])
outlier_df.sort_values('Количество выбросов', ascending=False)
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] >= lower) & (data[column] <= upper)]


# Применяем к числовым признакам
for col in ['Age', 'CreditScore', 'NumOfProducts']:
    df = remove_outliers_iqr(df, col)
    
# ваш код

# Удаляем 'Surname', т.к. она уникальна и бесполезна для модели
df = df.drop(columns=['Surname'], errors='ignore')

# Преобразуем 'Gender' в бинарный формат
if 'Gender' in df.columns:
    df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

# One-hot кодирование для 'Geography'
if 'Geography' in df.columns:
    df = pd.get_dummies(df, columns=['Geography'], drop_first=True)

from sklearn.preprocessing import StandardScaler

num_features = ['CreditScore', 'Age', 'Tenure',
                'Balance', 'NumOfProducts', 'EstimatedSalary']

# Пересоздаем X и y после обработки
X = df.drop('Exited', axis=1)
y = df['Exited']

# Разбиваем заново, так как изменился df
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

# ваш код

from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=1000, random_state=42)
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

# ваш код

# Создаем модель логистической регрессии с регуляризацией
model2 = LogisticRegression(
    penalty='l2', C=1.0, solver='liblinear', max_iter=1000)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("\nМодель 1 (без регуляризации):")
print("Accuracy:", accuracy_score(y_test, y_pred1))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred1))
print("Classification Report:\n", classification_report(
    y_test, y_pred1, zero_division=0))

print("\n"+"="*40+"\n")

print("Модель 2 (с L2-регуляризацией):")
print("Accuracy:", accuracy_score(y_test, y_pred2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred2))
print("Classification Report:\n", classification_report(
    y_test, y_pred2, zero_division=0))

Модель 1 (логистическая регрессия без явной регуляризации):
Accuracy: 0.8333989759747932
Confusion Matrix:
 [[24844  1179]
 [ 4320  2664]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90     26023
           1       0.69      0.38      0.49      6984

    accuracy                           0.83     33007
   macro avg       0.77      0.67      0.70     33007
weighted avg       0.82      0.83      0.81     33007



Модель 2 (логистическая регрессия с L2-регуляризацией):
Accuracy: 0.8334898657860453
Confusion Matrix:
 [[24847  1176]
 [ 4320  2664]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90     26023
           1       0.69      0.38      0.49      6984

    accuracy                           0.83     33007
   macro avg       0.77      0.67      0.70     33007
weighted avg       0.82      0.83      0.81     33007

